# Introduction and main quantities

The quantities are expressed in the following units:
-   density in $10^{19}$ $m^{-3}$
-   temperature in $keV$
-   current in $MA$ 
-   power in $MW$
-   distances in $m$
-   magnetic field in $T$
-   time in $s$

Reminder:
- safety factor at $r=a$: $q_a$
- inverse of the aspect ratio: $\varepsilon=R/a$
- elongation: $\kappa=b/a$
- triangularity: $\delta$


# Scaling Laws

In [1]:
%matplotlib widget 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.constants import pi, e, k, mu_0

In [2]:
# Constant terms
C_loss = 6 * pi**2 * 1e19 * 1e-3 * e # for W in MW.s
C_fus = 17.59 * e * 1.18e-24 * 1e38 * pi**2 / 2  # for P_fus in MW
C_fus = 9.51*pi**2*e*1e14 # ?? Yanick
C_beta = 4e2 * mu_0 * 1e19 * 1e3 * e  # beta expressed in %
C_n = 10/pi  # for n in 10^19 m^-3
C_I = 2 * pi * 1e-6 / mu_0  # for I in MA
ratio = 4.94  # ratio Pfus/Palpha (=5 w/o relativistic effects; =4.94 w/ relativistic effects)

In [3]:
print(f'C_loss={C_loss}')  # OK
print(f'C_fus={C_fus}')  # OK
print(f'C_beta={C_beta}')  # OK
print(f'C_n={C_n}')  # OK
print(f'C_I={C_I}')  # OK

C_loss=0.09487709656782095
C_fus=0.0015038019805999618
C_beta=0.8053418082653762
C_n=3.183098861837907
C_I=4.999999999999999


In [4]:
# Additional corrections from Johner FST 2001
def _C_I(epsilon=0.323, kappa=1.7, delta=0.33):
    return C_I*(1.17 - 0.65*epsilon)/(1 - epsilon**2)**2 * \
        0.5*(1 + kappa**2 * (1 + 2*delta**2 - 1.2*delta**3))

# Additional parameters
falpha = 0.1  # fraction of alpha particles
fp     = 1.5  # peaking factor of the temperature profile (<p^2> = fp <p>^2)
thetai = 1/1.25  # Ratio of Te & Ti temperatures

# Modified parameters
def _Meff(falpha=0.1):
    " Average ion mass [Atomic Mass Unit] "
    return (5 - 2*falpha)/(2*(1 - falpha))

def _Cfus(falpha=0.1, thetai=0.8, fp=1.5):
    return C_fus * (1 - 2*falpha)**2 * thetai**2 * fp

def _Closs(falpha=0.1, thetai=0.8):
    return C_loss * (1 + thetai - thetai*falpha)/2 

def _Cbeta(falpha=0.1, thetai=0.8):
    return C_beta * (1 + thetai - thetai*falpha)/2

Meff = _Meff()
C_loss = _Closs()
C_fus = _Cfus()
C_beta = _Cbeta()
C_I = _C_I()

print(f'Meff={Meff}')
print(f'C_loss={C_loss}')
print(f'C_fus={C_fus}')
print(f'C_beta={C_beta}')
print(f'C_I={C_I}') 

Meff=2.6666666666666665
C_loss=0.08159430304832602
C_fus=0.000923935936880617
C_beta=0.6925939551082235
C_I=13.148529478957025


In [5]:
def nTtau_fromQ(Q=10, lambd=ratio):
    " n.T.tau_e from eq (2.18) "
    return C_loss/C_fus * Q / (1 + Q/lambd)

# TODO : the following is not usefull later
def nTtau(M=2.7, kappa=1.7, epsilon=0.323, qa=3, n_N=0.85, R=6.2, B=5.3, beta_N=1.8):
    " n.T.tau_e from eq (2.20) "
    C_SL = 0.0562
    # (n*T*Tau)^0.31 
    _nTTau = C_SL * C_n**0.41 * C_I **0.96 * C_beta**0.38 * M**0.19 * kappa**0.09 * \
             epsilon**0.68 * qa**(-0.96) * n_N**0.41 * R**0.42 * B**0.73 * beta_N**(-0.38)
    return _nTTau**(1/0.31)

In [6]:
# check from §2.8
nTtau_fromQ(Q=10)  # ~300 for Q --> +oo

292.00771716378296

In [7]:
# beta_N expressions
def _beta_N1(P_DT=410, kappa=1.7, epsilon=0.323, qa=3, R=6.2, B=5.3):
    " beta_N from expression (2.10) "
    beta_N = qa * C_beta * np.sqrt(P_DT /(R**3 * B**4))  / \
                     np.sqrt(C_fus * C_I**2 * kappa * epsilon**4)
    return beta_N

def _beta_N2(Q=10, lambd=ratio, M=2.7, kappa=1.7, epsilon=0.323, qa=3, n_N=0.85, beta_N=1.8, R=6.2, B=5.3):
    " beta_N from expression (2.20)"
    _nTtau = nTtau_fromQ(Q, lambd)
    numer = _nTtau**0.31
    C_SL = 0.0562
    denom = C_SL * C_n**0.41 * C_beta**0.96 * M**0.19 * kappa**0.09 * epsilon**0.68 * \
            qa**(-0.96) * n_N**0.41 * R**0.42 * B**0.73
    return (numer/denom)**(-1/0.38) * 1e5 # why x 1e5 to have the correct scale?


def _beta_N3(Q=10, lambd=ratio, M=2.7, kappa=1.7, epsilon=0.323, qa=3, n_N=0.85, beta_N=1.8, R=6.2, B=5.3, IPB98=True):
    if IPB98:  # Case ITER IPB98(y,2): ELMy H-mode
        C_SL = 0.0562 # constant coef
        aM = 0.19 # average mass
        aK = 0.78 # elongation kappa
        aE = 0.58 # inverse aspect ratio epsilon
        aN = 0.41 # density
        aI = 0.93 #  plasma current
        aR = 1.97 #  major radius
        aB = 0.15 #  magnetic field
        aP = -0.69 #  net power
    else:  # [Sips et al., NF 58 (2018) 126010]
        C_SL = 0.028 #  constant coef
        aM = 0.14 #  average mass
        aK = 0.75 #  elongation kappa
        aE = 0.30 #   inverse aspect ratio epsilon
        aN = 0.49 #   density
        aI = 0.83 #   plasma current
        aR = 1.51 #  major radius
        aB = 0.07 #   magnetic field
        aP = -0.55 #   net power
    abetaN = 1+2*aP
    return (C_SL * C_n**aN * C_fus**(1+aP) / C_loss)**(-1/abetaN) * \
        C_I**(-1 - (aN+aI)/abetaN) * C_beta * \
        (M**aM * kappa**(aK+aP) * epsilon**(aE+2*aI+1+4*aP))**(-1/abetaN) * \
        qa**(1+(aN+aI)/abetaN) * (n_N**aN * R**(aR+aI-aN+3*aP))**(-1/abetaN) * \
        B**(-2-(aB+aN+aI)/abetaN) * ((1+Q/lambd)/Q)**(-(1+aP)/abetaN)

In [8]:
_beta_N1()

1.784534657762421

In [9]:
_beta_N2()

1.6435813458224864

In [10]:
_beta_N3(IPB98=True)

1.8285413942173556

In [11]:
_B = np.linspace(3, 9, 101)
_R = np.linspace(2, 10, 101)
_RR, _BB = np.meshgrid(_R, _B)

beta_N1 = _beta_N1(B=_BB, R=_RR)
beta_N2 = _beta_N2(B=_BB, R=_RR)
beta_N3 = _beta_N3(B=_BB, R=_RR, IPB98=False)

In [45]:
fig, ax = plt.subplots(figsize=(7,5))
c1=ax.contour(_RR, _BB, beta_N1, levels=[1.4, 1.6, 1.8,  2, 2.2],
             alpha=0.5, cmap='copper')
ax.clabel(c1, inline=1, fontsize=10, fmt='%.1f')
c2=ax.contour(_RR, _BB, beta_N2, levels=[1.4, 1.6,  1.8,  2, 2.2], 
              linestyles='dashed', alpha=0.8)
ax.clabel(c2, inline=1, fontsize=10, fmt='%.1f')
ax.set_xlabel('R [m]')
ax.set_ylabel('B [T]')
ax.set_title(r'plain lines: $\beta_N(R,B)$ from (2.10)'+' \n'+ r' dashed lines: $\beta_N(R,B)$ from (2.20)')

# determine the points where both beta_N are equals to the target (common points)
# from the contour lines
# beta_N_target = 1.8 -> 3rd line on contour
RB_sol1 = c1.allsegs[2][0]
RB_sol2 = c2.allsegs[2][0]
B_sol1 = RB_sol1[:,1]
B_sol2 = np.interp(RB_sol1[:,0], RB_sol2[:,0], RB_sol2[:,1])  # interpolate to be able to find the nearest value 

R_sol = RB_sol1[np.argmin(np.abs(B_sol1 - B_sol2)), 0]
B_sol = RB_sol1[np.argmin(np.abs(B_sol1 - B_sol2)), 1]

ax.plot(R_sol, B_sol, '.', ms=20, color='k')

ax.annotate(fr'(R,B) for $\beta_N=1.8$: ({R_sol:.2f} m, {B_sol:.2f} T)', 
            xy=(R_sol+.1, B_sol+.1), xytext=(5,8),
            arrowprops={'arrowstyle': '->'}, va='center')

fig.savefig('beta_N.png', dpi=150)

FigureCanvasNbAgg()

Since there is two equations $\beta_{N,1}(R,B)$ and $\beta_{N,2}(R,B)$, we solve for $(R,B)$ 

In [13]:
# for each R, find the points B for which beta_N1 = beta_N2
from scipy.optimize import fsolve

def diff_beta_N12(B, R):
    return _beta_N1(R=R, B=B) - _beta_N2(R=R, B=B)

def diff_beta_N13(B, R):
    return _beta_N1(R=R, B=B) - _beta_N3(R=R, B=B, IPB98=True)

B_sol12 = []
B0 = 5
for R in _R:
    B_sol12.append(fsolve(diff_beta_N12, B0, args=R))
B_sol12 = np.array(B_sol12).squeeze()

B_sol13 = []
for R in _R:
    B_sol13.append(fsolve(diff_beta_N13, B0, args=R))
B_sol13 = np.array(B_sol13).squeeze()

In [50]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
c=ax.plot_surface(_RR, _BB, beta_N1, alpha=0.6, cmap=cm.viridis, vmin=0, vmax=3)
ax.plot_surface(_RR, _BB, beta_N2, alpha=0.6, cmap=cm.viridis, vmin=0, vmax=3)
ax.set_xlabel('R [m]')
ax.set_ylabel('B [T]')
ax.set_zlabel(r'$\beta_N$')
ax.set_aspect('equal')
ax.set_zlim(0, 5)
ax.plot(_R, B_sol12, _beta_N1(R=_R, B=B_sol12), color='r', lw=2)
ax.set_xlim(2, 10)
ax.set_ylim(3, 9)
ax.view_init(azim=-45, elev=25)
fig.colorbar(c, shrink=0.7)
fig.tight_layout()
ax.set_title(r'$\beta_N$(R,B) from eq.(2.10) and (2.20)')
fig.savefig('beta_N_3D.png', dpi=150)

FigureCanvasNbAgg()

In [15]:
# relative distance between the two surfaces
delta_beta_N12 = 2*np.abs(beta_N1 - beta_N2)/(beta_N1 + beta_N2)
delta_beta_N13 = 2*np.abs(beta_N1 - beta_N3)/(beta_N1 + beta_N3)

In [16]:
fig, ax = plt.subplots()
c=ax.pcolor(_R, _B, diff_beta_N12(_BB, _RR), cmap='PiYG', vmin=-10, vmax=10)
fig.colorbar(c)
ax.set_ylim(3,9)
ax.set_xlim(2, 10)
ax.plot(_R, B_sol12)
#ax.plot(_R, B_sol13)
ax.set_xlabel('R [m]')
ax.set_ylabel('$B_0$ [T]')
ax.set_title(r'$\Delta\beta_N(R,B_0)$')

FigureCanvasNbAgg()

Text(0.5, 1.0, '$\\Delta\\beta_N(R,B_0)$')

In [17]:
# R^a B^b expressions
def RB1(Q=10, lambd=ratio, M=2.7, kappa=1.7, epsilon=0.323, qa=3, n_N=0.85, beta_N=1.8):
    " return R**0.42 * B**0.73 "
    C_SL = 0.0562 # constant coef
    # from (n*T*Tau)^0.31
    _temp = C_SL * C_n**0.41 * C_I **0.96 * C_beta**0.38 * \
            M**0.19 * kappa**0.09 * epsilon**0.68 * qa**(-0.96) * n_N**0.41 * beta_N**(-0.38)
        
    return (C_loss/C_fus * Q/(1+Q/lambd))**0.31 / (_temp)

def RB2(P_DT=410, kappa=1.7, epsilon=0.323, qa=3, beta_N=1.8):
    " return R**3 * B**4 "
    return P_DT / (C_beta**(-2) * C_fus * C_I**2 * qa**(-2) * kappa * epsilon**(4) * beta_N**(2))

# Retrieving ITER Parameters

## ITER data:
-   aspect ratio  = 3.1
-   elongation    = 1.85
-   triangularity = 0.33
-   q95           = 3

The ITER target are:
-   Fusion power  = 500 MW
-   Fusion gain   = 10 

In [18]:
Q_target = 10
Pfus_target = 410

beta_N_target = 1.8
n_N_target = 0.85
qa_target = 3
kappa_target = 1.7
epsilon_target = 0.323

In [19]:
RB1_target = RB1(Q_target, 
                lambd=ratio, M=Meff, kappa=kappa_target, 
                epsilon=epsilon_target, qa=qa_target, 
                n_N=n_N_target, beta_N=beta_N_target)

RB2_target = RB2(Pfus_target, 
                kappa=kappa_target, epsilon=epsilon_target, 
                qa=qa_target, beta_N=beta_N_target)

print(f'RB1_target={RB1_target}')
print(f'RB2_target={RB2_target}')

RB1_target=40.823420662464116
RB2_target=184834.7014937577


In [20]:
R_ITER = 6.2
B_ITER = 5.3

b1_ITER = R_ITER**0.42 * B_ITER**0.73
b2_ITER = R_ITER**3 * B_ITER**4
print(f'ITER: b1={b1_ITER}, b2={b2_ITER}')

ITER: b1=7.269857437594987, b2=188052.2555768


In [21]:
# let's do the opposite : what should be the best set of parameters
# in order to obtain ITER values of (R,B)
def obj_fun(x):
    # fixed parameters
    lambd = ratio

    # target (ITER)
    beta_N_target = 1.7
    
    # parameters to find
    R, B = x 
    
    beta_N1 = _beta_N1(R=R, B=B)
    beta_N2 = _beta_N3(R=R, B=B, IPB98=False)
    #beta_N2 = _beta_N3(R=R, B=B)

    # return scalar objective function
    return np.sqrt((beta_N_target - beta_N1)**2 + (beta_N_target - beta_N2)**2)


In [22]:
R0, B0 = 5, 4
x0 = [R0, B0]

x = minimize(obj_fun, x0)
x

      fun: 7.689875016042648e-09
 hess_inv: array([[ 5.72169354e-08, -4.83801408e-08],
       [-4.83801408e-08,  5.37779199e-08]])
      jac: array([1.19412642, 0.89152643])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 648
      nit: 49
     njev: 159
   status: 2
  success: False
        x: array([ 1.99153383, 12.72673209])